In [51]:
import os
import geopandas
import numpy as np
import googlemaps
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from datetime import datetime
import time
import re

###############################
# Inputs

# Destination coordinates (lat, long)
#destination = (55.6783,12.5864)

destination = (42.376970, -71.105210)

west = -71.118
east = -71.007
north = 42.425
south = 42.325


#sizelat = 0.1
#sizelong = 0.1

# Grid corners coordinates (lat, long)
#corner1 = (55.86484,12.0726363)
#corner2 = (55.550041,12.6874933)

# Grid resolution (lat, long)
resolution = (0.001, 0.001)

# Arrival time (yyyy, m, d, h, m)
arrivaldate = datetime(2022, 5, 25, 12, 0)

# Travel mode ('bicycling', 'driving', 'transit', 'walking')
travelmode = "transit"

# Time delay between queries (s)
dt = 0.1

# Text file with google maps API key
apipath = os.path.join(os.getcwd(),'apikey.txt')

#onwaterapikey = '9PgvayVQfnqr152V2aQQ'

# Coastline data
coastdata = os.path.join(os.getcwd(),'data','coastlines','OUTLINE25K_POLY_dd.shp')

coastrow = 0

# Coastline data
#coastdata = os.path.join(os.getcwd(),'data','coastlines','MassBay_Source','MassBay_Source.shp')

#coastrow = 27

# Coastline data
#coastdata = os.path.join(os.getcwd(),'data','coastlines','zealand_amager.shp')

###############################
# Functions

# Convert time string to minutes
def timestr2int(timestr):
    # Find and convert hours
    hours = re.findall('\d+ hour', timestr)
    if len(hours) == 0:
        hours = 0
    else:
        hours = int(hours[0][:-5])
        
    # Find and convert minutes
    mins = re.findall('\d+ min', timestr)
    if len(mins) == 0:
        mins = 0
    else:
        mins = int(mins[0][:-4])

    # Total time in min
    return hours*60 + mins

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

###############################
# Main script

# Data path
dpath = os.path.join(os.getcwd(),'data')

# Load coastlines
print('Loading coastline data ...')
longlatcoasts = geopandas.read_file(coastdata)

#trim coastline to bounding box
nw_corner = (west, north)
se_corner = (east, south)
ne_corner = (east, north)
sw_corner = (west, south)

polys1 = geopandas.GeoSeries([Polygon([nw_corner, ne_corner,se_corner, sw_corner])])

square = geopandas.GeoDataFrame({'geometry': polys1})

coastclip = geopandas.overlay(longlatcoasts.loc[[coastrow]],square, how='intersection')


# Grid coordinates
lats = np.arange(west,east,
                 np.sign(east-west)*resolution[0])
longs = np.arange(north,south,
                 np.sign(south-north)*resolution[1])

print("Done!")

Loading coastline data ...
Done!


In [48]:
addresses = []
# Loop
print('Computing shorelines...')
for ii, long in enumerate(longs):
    for jj, lat in enumerate(lats):
        # Print progress
        if not (ii*len(lats)+jj)%1000:
            print('%i/%i' %(ii*len(lats)+jj,len(lats)*len(longs)))
        
        # Check if the coast polygon contains origin
        point = Point(lat,long)
        #if coastclip.iloc[0]["geometry"].contains(point):
        if longlatcoasts.iloc[coastrow]["geometry"].contains(point):
            addresses.append(('%.6f,%.6f') %(long,lat)) 
                  
print(len(addresses))

Computing shorelines...
0/11100
1000/11100
2000/11100
3000/11100
4000/11100
5000/11100
6000/11100
7000/11100
8000/11100
9000/11100
10000/11100
11000/11100
9538


In [49]:
# Load API key
with open(apipath) as f:
    api_key = f.readline()
    f.close

# Googlemaps sign in
gmaps = googlemaps.Client(key=api_key)

# Create data file
fname = 'bostonbig_traveldata.csv'
np.savetxt(os.path.join(dpath,fname),[],header='lat,long,time(min)')


# Query google maps distance matrix 25 addresses at a time
print('Collecting data...')
for address_chunk in chunker(addresses,25):
    result = gmaps.distance_matrix(address_chunk, destination,mode=travelmode,arrival_time=arrivaldate)
    for add, res in zip(address_chunk,result['rows']):
        status = res['elements'][0]['status']
        if status != 'OK':
            continue
        
        timestr = res['elements'][0]['duration']['text']
        T = timestr2int(timestr)
        # Append row to file
        with open(os.path.join(dpath,fname),'a') as fd:
            fd.write(add+',%i\n'%(T))

    # Pause
    time.sleep(dt)

print('Done!')

Done!
